In [ ]:
import numpy as np
import tensorflow as tf
from IPython.display import clear_output, display
from tensorflow.python.saved_model import tag_constants
import sys
sys.path.insert(0, '..')
from celeba_reader import CelebaSequence

#### Prepare testing dataset

In [ ]:
paths = {
    'img_path': '/path/to/img_align_celeba',
    'attrs_path': 'list_attr_celeba.csv',
    'partition_path': 'list_eval_partition.csv'
}
batch = 100
test_data = CelebaSequence(partition=2, batch=batch, **paths)

#### Measure test accuracy

In [ ]:
def show(s):
    clear_output()
    display(s)

graph = tf.Graph()
with graph.as_default():
    with tf.Session() as sess:
        # load model
        tf.saved_model.loader.load(sess, [tag_constants.SERVING], './saved_model')
        
        # get input placeholders and output node
        images_batch = graph.get_tensor_by_name('Placeholder:0')
        labels = graph.get_tensor_by_name('Placeholder_1:0')
        classifier = graph.get_tensor_by_name('Reshape_1:0')

        # make predictions for test data and measure accuracy
        correct, total = 0., 0.
        for x, y in test_data:
            predicted = sess.run(classifier, feed_dict={images_batch: x, labels: y})
            predicted[predicted >= 0.5] = 1
            predicted[predicted < 0.5 ] = 0
            correct = correct + np.sum(np.equal(y, predicted))
            total = total + len(y)
            show(f"{correct}/{total}")
        show(f"accuracy={correct/total:.2f}")

If you want to visualize loaded graph, add:
```python
tf.summary.FileWriter( './logs/1/train', graph)
```
Then run tensorboard in terminal, providing the directory with the saved graph:
```
$ tensorboard --logdir ./logs/1
```